# 파이썬 복습 세션
세션에서 배운 내용을 파이썬으로 구현해보자!  
  



## 목차
1. CAPM  
2. Valuation
3. RNP  

## 1. CAPM
### 1-1. 간단한 모형으로 CML 그려보기
시작 전 왼쪽 맨 아래의 파일 아이콘을 눌러 코랩 드라이브를 마운트 해주세요

In [ ]:
!pip install --upgrade pandas-datareader

In [ ]:
#thx to financeandpython.com

import matplotlib.pyplot as plt #import needed packages


def CML(rf, rm, marketSD, label): #함수 정의
  portfolioSD = [(marketSD*x)/25 for x in range(51)] #포트폴리오 표준편차
  portfolioReturns = [rf+((rm - rf)/marketSD)*x for x in portfolioSD] #리스크프리미엄
  plt.plot(portfolioSD, portfolioReturns, label=label)
  plt.xlabel("Portfolio Standard Deviation")
  plt.ylabel("Portfolio Return")
  plt.title("Capital Market Line")
  plt.plot(marketSD, rm, "ro")

In [ ]:
CML(.02, .08, .35,"CML")
plt.legend()
plt.show()

In [ ]:
#무위험이자율의 증가
CML(.02, .08, .35,"Original")
CML(.04, .08, .35,"Riskfree Increase")  #무위험이자율 2%p 증가
plt.legend()
plt.show()

In [ ]:
#시장이자율이 증가
CML(.02, .08, .35,"Original")
CML(.02, .10, .35,"Market Increase")  #시장이자율 2%p 증가
plt.legend()
plt.show()

In [ ]:
#두 이자율 모두 증가
CML(.02, .08, .35,"Original")
CML(.04, .10, .35,"Increase")  #모든 이자율 2%p 증가
plt.legend()
plt.show()

### 1-2. 데이터를 통해 실제 SML 그려보기

In [ ]:
#박기영 교수님 화폐금융론 강의

# import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%config InlineBackend.figure_format = 'retina'   # for retina display in Mac 

In [ ]:
pd.set_option('display.precision', 3) #소수점 출력의 정밀도

In [ ]:
df = pd.read_csv('flatSML.csv') #외부 데이터 파일 가져오기
df.head()

In [ ]:
date = pd.date_range(start = '1990-01', end='2015-01',freq='M') #원하는 범위의 데이터만 월 단위로 지정
df['date'] = date #가져온 데이터프레임에 지정한 날짜 배당
df.set_index('date',inplace=True) #말 그대로 index 설정
df.head()

In [ ]:
Er_M = df['r_m'].mean() #시장이자율의 평균
rf = df['r_f'].mean() #무위험이자율의 평균
riskpremium = Er_M - rf #리스크 프리미엄
print('average market return is {}'.format(Er_M))
print('riskfree rate is {}'.format(rf))

In [ ]:
summary = df.agg([np.mean, np.std]) #안에 들어있는 함수를 적용해서 계산
summary = summary.T #transpose(다음 줄의 열 이름 지정을 위해)
summary.columns = ['avg. returns','std']  #열 이름 지정
summary

In [ ]:
summary['Sharpe ratio'] = (summary['avg. returns']-rf)/summary['std'] #새로운 열을 기존 열들에 대한 계산으로 지정
summary

In [ ]:
#베타값 계산

storage_beta = []
var_rM = df['r_m'].var() #시장이자율의 분산

for col in df.columns:
  covar = np.cov(df[col],df['r_m'])[0,1] #공분산
  beta_estimate = covar/var_rM
  storage_beta.append(beta_estimate)

In [ ]:
storage_beta

In [ ]:
summary['beta'] = pd.Series(storage_beta, index=summary.index)  #열 지정을 통해 계산한 베타값을 기존 데이터프레임에 넣음

In [ ]:
results = summary.T #뒤의 과정을 위한 트랜스포즈
results

In [ ]:
forgraph = summary[2:]  #summary의 3행(파이썬은 숫자를 0부터 셈!)부터 가져옴
forgraph

In [ ]:
xval = np.linspace(0,2,200)
yval = rf + riskpremium*xval

plt.plot(xval,yval)
plt.scatter(forgraph['beta'],forgraph['avg. returns'])
plt.grid()
plt.title("flat SML")
plt.xlabel('estimated beta')
plt.ylabel('annualized avg. returns')
plt.show()

### 3. Another way of CAPM
특정 시점의 시장의 return과 특정 주식 종목의 return 사이의 관계   
https://towardsai.net/p/l/capital-assets-pricing-model-capm%E2%80%8A-%E2%80%8Ausing-python

In [ ]:
#thx to towardsai.net

from scipy import stats
import pandas as pd
import pandas_datareader as web
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#주가 정보 수집 기간 설정
start = datetime.datetime(2020,1,1)
end = datetime.datetime(2021,1,1)

In [ ]:
df_samsung = web.DataReader("005930.KS","yahoo",start,end) 
#야후 파이낸스에서 원하는 주가 정보를 가져올 수 있는 함수
df_samsung.head()

In [ ]:
df_gs = web.DataReader("078930.KS","yahoo",start,end)
df_gs.head()

In [ ]:
df_samsung['Close'].plot(label = 'Samsung', figsize=(10,8)) #종가 그래프 그리기
df_gs['Close'].plot(label = 'GS')
plt.legend()

In [ ]:
#종목의 첫날 종가에 대한 업다운

df_samsung['Cumu'] = df_samsung['Close']/df_samsung['Close'].iloc[0] #iloc은 인덱스의 '위치'로 데이터 추출
df_gs['Cumu'] = df_gs['Close']/df_gs['Close'].iloc[0]

In [ ]:
df_samsung['Cumu'].plot(label = 'Sansung', figsize=(10,8))
df_gs['Cumu'].plot(label = 'GS')
plt.legend()

In [ ]:
df_samsung['daily_ret'] = df_samsung['Close'].pct_change()  #수익률 구하기
df_gs['daily_ret'] = df_gs['Close'].pct_change()

plt.scatter(df_samsung['daily_ret'],df_gs['daily_ret'])

## 2. Valuation - DCF Model  
코드 출처: https://towardsdatascience.com/discounted-cash-flow-with-python-f5103921942e  
  




모델 구동에 앞서서 api키 발급을 위해  
https://site.financialmodelingprep.com/  
에 접속해서 회원가입 및 api키 발급을 완료합시다.  
키 발급이 완료되면 demo변수에 지정해줄 것!  
  
  여기서 api키란?  
  사이트에 접속하는 것을 문을 통해 방에 들어가는 것으로 비유한다면 열쇠가 필요한 문이 있기도 한데, 그 문의 열쇠 역할을 하는 코드  
  공동사용 절대 안됨! (열쇠보다는 지문인식같은 느낌)

In [ ]:
#Estimating Future Revenue Growth
#기업 정보 가져오기

import requests
import numpy as np
import pandas as pd
company = 'GOOG'
demo = 'your api key' #api키는 여기에
IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={demo}').json()
print(IS)

In [ ]:
count = 0

#미래의 매출을 예측하기 위해 수익 성장 정보를 얻기
revenue_g = [] #revenue를 담을 빈 list

for item in IS:
  if count < 4: #반복횟수
    #print(item)
    revenue_g.append(item['revenue'])  #저 위의 revenue 값을 빈 리스트에 추가
    count += 1
    
revenue_g = (revenue_g[0] - revenue_g[1]) /revenue_g[0] #리스트를 기반으로 revebue 성장 구하기
print(revenue_g)

In [ ]:
#Income Statement(손익계산서), 대차대조표(Balance Sheet) 구하기

#net income 구하기
net_income = IS[0]['netIncome']
BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={demo}').json()

#get income statement as % of revenue for future predictions and forecast 5 next IS years
#손익계산서를 미래 예측에 대한 수익의 비율로 가죠와서 다음 5년을 예측함

income_statement = pd.DataFrame.from_dict(IS[0],orient='index')  #첫 기의 데이터를 데이터프레임으로 만듦
#The [7:31] below get rids of not needed items coming from the API
income_statement = income_statement[7:31]

income_statement.columns = ['current_year']
print(income_statement)


In [ ]:
#위 표의 각 내용을 매출에 대한 비율로 표시
income_statement['as_%_of_revenue'] = income_statement.iloc[1:].astype(float)  / float(income_statement.iloc[1])

#forecasting 5 next years income statement
#다음해 = 전년도 수익 * (1+수익성장률) * 수익에 대한 비율
income_statement['next_year'] =  (income_statement['current_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
income_statement['next_2_year'] =  (income_statement['next_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
income_statement['next_3_year'] =  (income_statement['next_2_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
income_statement['next_4_year'] =  (income_statement['next_3_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
income_statement['next_5_year'] =  (income_statement['next_4_year']['revenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue']

In [ ]:
print(income_statement)

In [ ]:
#Get Balance sheet as a percentage of revenue
#아까 손익계산서랑 같은 과정
balance_sheet = pd.DataFrame.from_dict(BS[0],orient='index')
balance_sheet = balance_sheet[8:-2]
balance_sheet.columns = ['current_year']

print(balance_sheet)

In [ ]:
print(float(income_statement['current_year'].iloc[1]))

In [ ]:
#손익계산서랑 과정 같음

balance_sheet['as_%_of_revenue'] = balance_sheet.iloc[3:].astype(float) / float(income_statement['current_year'].iloc[1])
#대차대조표의 4행부터를 손익계산서의 current year의 revenue로 나눔

#forecasting the next 5 years Balance Sheet.
balance_sheet['next_year'] =  income_statement['next_year'] ['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['next_2_year'] =  income_statement['next_2_year'] ['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['next_3_year'] =  income_statement['next_3_year']['revenue'] * balance_sheet['as_%_of_revenue'] 
balance_sheet['next_4_year'] =  income_statement['next_4_year']['revenue']  * balance_sheet['as_%_of_revenue'] 
balance_sheet['next_5_year'] =  income_statement['next_5_year']['revenue'] * balance_sheet['as_%_of_revenue']

In [ ]:
print(balance_sheet)

In [ ]:
#Forecasting Future Cash Flow From Operations

CF_forecast = {}
CF_forecast['next_year'] = {}

#순이익
CF_forecast['next_year']['netIncome'] = income_statement['next_year']['netIncome']

#감가상각
CF_forecast['next_year']['inc_depreciation'] = income_statement['next_year']['depreciationAndAmortization'] - income_statement['current_year']['depreciationAndAmortization']

#채권
CF_forecast['next_year']['inc_receivables'] = balance_sheet['next_year']['netReceivables'] - balance_sheet['current_year']['netReceivables']

#재고
CF_forecast['next_year']['inc_inventory'] = balance_sheet['next_year']['inventory'] - balance_sheet['current_year']['inventory']

#채무
CF_forecast['next_year']['inc_payables'] = balance_sheet['next_year']['accountPayables'] - balance_sheet['current_year']['accountPayables']

#영업활동 현금흐름
CF_forecast['next_year']['CF_operations'] = CF_forecast['next_year']['netIncome'] + CF_forecast['next_year']['inc_depreciation'] + (CF_forecast['next_year']['inc_receivables'] * -1) + (CF_forecast['next_year']['inc_inventory'] *-1) + CF_forecast['next_year']['inc_payables']

#자본적 지출
CF_forecast['next_year']['CAPEX'] = balance_sheet['next_year']['propertyPlantEquipmentNet'] - balance_sheet['current_year']['propertyPlantEquipmentNet'] + income_statement['next_year']['depreciationAndAmortization']

#잉여현금흐름
CF_forecast['next_year']['FCF'] = CF_forecast['next_year']['CAPEX'] + CF_forecast['next_year']['CF_operations']

In [ ]:
#그 이후의 해도 이런 식으로 예측
CF_forecast['next_2_year'] = {}
CF_forecast['next_2_year']['netIncome'] = income_statement['next_2_year']['netIncome']
CF_forecast['next_2_year']['inc_depreciation'] = income_statement['next_2_year']['depreciationAndAmortization'] - income_statement['next_year']['depreciationAndAmortization']
CF_forecast['next_2_year']['inc_receivables'] = balance_sheet['next_2_year']['netReceivables'] - balance_sheet['next_year']['netReceivables']
CF_forecast['next_2_year']['inc_inventory'] = balance_sheet['next_2_year']['inventory'] - balance_sheet['next_year']['inventory']
CF_forecast['next_2_year']['inc_payables'] = balance_sheet['next_2_year']['accountPayables'] - balance_sheet['next_year']['accountPayables']
CF_forecast['next_2_year']['CF_operations'] = CF_forecast['next_2_year']['netIncome'] + CF_forecast['next_2_year']['inc_depreciation'] + (CF_forecast['next_2_year']['inc_receivables'] * -1) + (CF_forecast['next_2_year']['inc_inventory'] *-1) + CF_forecast['next_2_year']['inc_payables']
CF_forecast['next_2_year']['CAPEX'] = balance_sheet['next_2_year']['propertyPlantEquipmentNet'] - balance_sheet['next_year']['propertyPlantEquipmentNet'] + income_statement['next_2_year']['depreciationAndAmortization']
CF_forecast['next_2_year']['FCF'] = CF_forecast['next_2_year']['CAPEX'] + CF_forecast['next_2_year']['CF_operations']
CF_forecast['next_3_year'] = {}
CF_forecast['next_3_year']['netIncome'] = income_statement['next_3_year']['netIncome']
CF_forecast['next_3_year']['inc_depreciation'] = income_statement['next_3_year']['depreciationAndAmortization'] - income_statement['next_2_year']['depreciationAndAmortization']
CF_forecast['next_3_year']['inc_receivables'] = balance_sheet['next_3_year']['netReceivables'] - balance_sheet['next_2_year']['netReceivables']
CF_forecast['next_3_year']['inc_inventory'] = balance_sheet['next_3_year']['inventory'] - balance_sheet['next_2_year']['inventory']
CF_forecast['next_3_year']['inc_payables'] = balance_sheet['next_3_year']['accountPayables'] - balance_sheet['next_2_year']['accountPayables']
CF_forecast['next_3_year']['CF_operations'] = CF_forecast['next_3_year']['netIncome'] + CF_forecast['next_3_year']['inc_depreciation'] + (CF_forecast['next_3_year']['inc_receivables'] * -1) + (CF_forecast['next_3_year']['inc_inventory'] *-1) + CF_forecast['next_3_year']['inc_payables']
CF_forecast['next_3_year']['CAPEX'] = balance_sheet['next_3_year']['propertyPlantEquipmentNet'] - balance_sheet['next_2_year']['propertyPlantEquipmentNet'] + income_statement['next_3_year']['depreciationAndAmortization']
CF_forecast['next_3_year']['FCF'] = CF_forecast['next_3_year']['CAPEX'] + CF_forecast['next_3_year']['CF_operations']
CF_forecast['next_4_year'] = {}
CF_forecast['next_4_year']['netIncome'] = income_statement['next_4_year']['netIncome']
CF_forecast['next_4_year']['inc_depreciation'] = income_statement['next_4_year']['depreciationAndAmortization'] - income_statement['next_3_year']['depreciationAndAmortization']
CF_forecast['next_4_year']['inc_receivables'] = balance_sheet['next_4_year']['netReceivables'] - balance_sheet['next_3_year']['netReceivables']
CF_forecast['next_4_year']['inc_inventory'] = balance_sheet['next_4_year']['inventory'] - balance_sheet['next_3_year']['inventory']
CF_forecast['next_4_year']['inc_payables'] = balance_sheet['next_4_year']['accountPayables'] - balance_sheet['next_3_year']['accountPayables']
CF_forecast['next_4_year']['CF_operations'] = CF_forecast['next_4_year']['netIncome'] + CF_forecast['next_4_year']['inc_depreciation'] + (CF_forecast['next_4_year']['inc_receivables'] * -1) + (CF_forecast['next_4_year']['inc_inventory'] *-1) + CF_forecast['next_4_year']['inc_payables']
CF_forecast['next_4_year']['CAPEX'] = balance_sheet['next_4_year']['propertyPlantEquipmentNet'] - balance_sheet['next_3_year']['propertyPlantEquipmentNet'] + income_statement['next_4_year']['depreciationAndAmortization']
CF_forecast['next_4_year']['FCF'] = CF_forecast['next_4_year']['CAPEX'] + CF_forecast['next_4_year']['CF_operations']
CF_forecast['next_5_year'] = {}
CF_forecast['next_5_year']['netIncome'] = income_statement['next_5_year']['netIncome']
CF_forecast['next_5_year']['inc_depreciation'] = income_statement['next_5_year']['depreciationAndAmortization'] - income_statement['next_4_year']['depreciationAndAmortization']
CF_forecast['next_5_year']['inc_receivables'] = balance_sheet['next_5_year']['netReceivables'] - balance_sheet['next_4_year']['netReceivables']
CF_forecast['next_5_year']['inc_inventory'] = balance_sheet['next_5_year']['inventory'] - balance_sheet['next_4_year']['inventory']
CF_forecast['next_5_year']['inc_payables'] = balance_sheet['next_5_year']['accountPayables'] - balance_sheet['next_4_year']['accountPayables']
CF_forecast['next_5_year']['CF_operations'] = CF_forecast['next_5_year']['netIncome'] + CF_forecast['next_5_year']['inc_depreciation'] + (CF_forecast['next_5_year']['inc_receivables'] * -1) + (CF_forecast['next_5_year']['inc_inventory'] *-1) + CF_forecast['next_5_year']['inc_payables']
CF_forecast['next_5_year']['CAPEX'] = balance_sheet['next_5_year']['propertyPlantEquipmentNet'] - balance_sheet['next_4_year']['propertyPlantEquipmentNet'] + income_statement['next_5_year']['depreciationAndAmortization']
CF_forecast['next_5_year']['FCF'] = CF_forecast['next_5_year']['CAPEX'] + CF_forecast['next_5_year']['CF_operations']

In [ ]:
#에측된 현금흐름을 Pandas로 바꿈

#예측한 현금흐름을 Pandas DataFrame에 넣기
CF_forec = pd.DataFrame.from_dict(CF_forecast,orient='columns')
#add below option to format the dataframe with thousand separators
pd.options.display.float_format = '{:,.0f}'.format
print(CF_forec)

In [ ]:
#자본 가격 예측

import pandas_datareader.data as web
import datetime
import requests

company = 'GOOG'
demo = 'your api key'  #여기에도 api키 입력!

#wacc(가중평균자본비용): 기업의 총자본에 대한 평균조달비용
#즉, 기업이 현재 보유 중인 사잔을 활용하여 자사의 주식가치를 유지하기 위해 벌어들여야 하는 수익률


#Interest coverage ratio = EBIT / interest expenses
def interest_coveraga_and_RF(company):
  IS= requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={demo}').json()
  EBIT= IS[0]['ebitda'] - IS[0]['depreciationAndAmortization'] 
  interest_expense = IS[0]['interestExpense']
  interest_coverage_ratio = EBIT / interest_expense
#RF
  start = datetime.datetime(2019, 7, 10)
        
  end = datetime.datetime(2020, 7, 10)
  Treasury = web.DataReader(['TB1YR'], 'fred', start, end) #국채
  RF = float(Treasury.iloc[-1])/100
  return [RF,interest_coverage_ratio]
  
    
#Cost of debt
#신용등급에 따른 이자
def cost_of_debt(company, RF,interest_coverage_ratio):
  if interest_coverage_ratio > 8.5:
    #Rating is AAA
    credit_spread = 0.0063
  if (interest_coverage_ratio > 6.5) & (interest_coverage_ratio <= 8.5):
    #Rating is AA
    credit_spread = 0.0078
  if (interest_coverage_ratio > 5.5) & (interest_coverage_ratio <=  6.5):
    #Rating is A+
    credit_spread = 0.0098
  if (interest_coverage_ratio > 4.25) & (interest_coverage_ratio <=  5.49):
    #Rating is A
    credit_spread = 0.0108
  if (interest_coverage_ratio > 3) & (interest_coverage_ratio <=  4.25):
    #Rating is A-
    credit_spread = 0.0122
  if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <=  3):
    #Rating is BBB
    credit_spread = 0.0156
  if (interest_coverage_ratio > 2.25) & (interest_coverage_ratio <=  2.5):
    #Rating is BB+
    credit_spread = 0.02
  if (interest_coverage_ratio > 2) & (interest_coverage_ratio <=  2.25):
    #Rating is BB
    credit_spread = 0.0240
  if (interest_coverage_ratio > 1.75) & (interest_coverage_ratio <=  2):
    #Rating is B+
    credit_spread = 0.0351
  if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <=  1.75):
    #Rating is B
    credit_spread = 0.0421
  if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <=  1.5):
    #Rating is B-
    credit_spread = 0.0515
  if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <=  1.25):
    #Rating is CCC
    credit_spread = 0.0820
  if (interest_coverage_ratio > 0.65) & (interest_coverage_ratio <=  0.8):
    #Rating is CC
    credit_spread = 0.0864
  if (interest_coverage_ratio > 0.2) & (interest_coverage_ratio <=  0.65):
    #Rating is C
    credit_spread = 0.1134
  if interest_coverage_ratio <=  0.2:
    #Rating is D
    credit_spread = 0.1512
  
  cost_of_debt = RF + credit_spread
  return cost_of_debt


def costofequity(company):
#RF
  start = datetime.datetime(2019, 7, 10)
  end = datetime.datetime(2020, 7, 10)
  Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
  RF = float(Treasury.iloc[-1])
  RF = RF/100
#Beta
  beta = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{company}?apikey={demo}')
  beta = beta.json() 
#여기서 json이란 key-value쌍으로 이루어진 데이터 오브젝트를 전달하기 위해 인간이 읽을 수 있는 텍스트를 사용하는 개방형 표준 포맷
  beta = float(beta['profile']['beta'])
#Market Return
  start = datetime.datetime(2019, 7, 10)
  end= datetime.datetime.today().strftime('%Y-%m-%d')
  SP500 = web.DataReader(['sp500'], 'fred', start, end)
      #Drop all Not a number values using drop method.
  SP500.dropna(inplace = True)
  SP500yearlyreturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[-252])-1
    
  cost_of_equity = RF+(beta*(SP500yearlyreturn - RF))
  return cost_of_equity


#effective tax rate and capital structure
def wacc(company):
  FR = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{company}?apikey={demo}').json()
  ETR = FR[0]['effectiveTaxRate']
  BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={demo}').json()
  Debt_to = BS[0]['totalDebt'] / (BS[0]['totalDebt'] + BS[0]['totalStockholdersEquity']) #부채/자산
  equity_to = BS[0]['totalStockholdersEquity'] / (BS[0]['totalDebt'] + BS[0]['totalStockholdersEquity']) #자본/부채
  WACC = (kd*(1-ETR)*Debt_to) + (ke*equity_to)
  return WACC


RF_and_IntCov = interest_coveraga_and_RF(company)
RF = RF_and_IntCov[0]
interest_coverage_ratio = RF_and_IntCov[1]
ke = costofequity(company)
kd = cost_of_debt(company,RF,interest_coverage_ratio)
wacc_company = wacc(company)
print('wacc of ' + company + ' is ' + str((wacc_company*100))+'%')

In [ ]:
#numpy financial이란 라이브러리 설치 및 immport
#npv 구하는 부분이 있는데 여기에서 사용할 것

!pip install numpy_financial
import numpy_financial as npf

In [ ]:
#Forecasted Free Cash Flows의 NPV 구하기


#FCF List of CFs for each year
FCF_List = CF_forec.iloc[-1].values.tolist() #위에서 계산한 예측현금흐름의 맨 오른쪽 열
npv = npf.npv(wacc_company,FCF_List) #NUMPY FINANCIAL로 NPV 구하기

In [ ]:
#기업의 영구가치 구하기

LTGrowth = 0.02
Terminal_value = (CF_forecast['next_5_year']['FCF'] * (1+ LTGrowth)) /(wacc_company  - LTGrowth)
Terminal_value_Discounted = Terminal_value/(1+wacc_company)**4
print(Terminal_value_Discounted)

In [ ]:
#목표가격 구하기

target_equity_value = Terminal_value_Discounted + npv
debt = balance_sheet['current_year']['totalDebt']
target_value = target_equity_value - debt
numbre_of_shares = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{company}?apikey={demo}').json()
numbre_of_shares = numbre_of_shares[0]['numberOfShares']
target_price_per_share = target_value/numbre_of_shares
target_price_per_share
print(company + ' forecasted price per stock is ' + str(target_price_per_share) )
print('the forecast is based on the following assumptions: '+ 'revenue growth: ' + str(revenue_g) + ' Cost of Capital: ' + str(wacc_company) )
print('perpetuity growth: ' + str(LTGrowth)  )

## 3. RNP - Brownian  
https://towardsdatascience.com/brownian-motion-with-python-9083ebc46ff0  

In [ ]:
#brownian class 정의

class Brownian():
    """
    A Brownian motion class constructor
    """
    def __init__(self,x0=0):
        """
        Init class
        """
        assert (type(x0)==float or type(x0)==int or x0 is None), "최초의 값은 실수 또는 None"
        
        self.x0 = float(x0)
    
    def gen_random_walk(self,n_step=100):
        """
        Generate motion by random walk
        
        Arguments:
            n_step: Number of steps
            
        Returns:
            A NumPy array with `n_steps` points
        """
        # Warning about the small number of steps
        if n_step < 30:
            print("WARNING! The number of steps is small. It may not generate a good stochastic process sequence!")
        
        w = np.ones(n_step)*self.x0 #값이 1로 채워진 n_step 차원의 배열
        
        for i in range(1,n_step):
            # Sampling from the Normal distribution with probability 1/2
            yi = np.random.choice([1,-1]) #np.random.choice(sample, sample 개수, 복원 추출 여부(True/False), 확률)
            #sample이 정수인 경우 정수 샘플링
            # Weiner process
            w[i] = w[i-1]+(yi/np.sqrt(n_step))
        
        return w
    
    def gen_normal(self,n_step=100):
        """
        Generate motion by drawing from the Normal distribution
        
        Arguments:
            n_step: Number of steps
            
        Returns:
            A NumPy array with `n_steps` points
        """
        if n_step < 30:
            print("WARNING! The number of steps is small. It may not generate a good stochastic process sequence!")
        
        w = np.ones(n_step)*self.x0
        
        for i in range(1,n_step):
            # Sampling from the Normal distribution
            yi = np.random.normal() #정규분포에서 추출
            # Weiner process
            w[i] = w[i-1]+(yi/np.sqrt(n_step))
        
        return w
    
    def stock_price(
                    self,
                    s0=100,
                    mu=0.2,
                    sigma=0.68,
                    deltaT=52,
                    dt=0.1
                    ):
        """
        Models a stock price S(t) using the Weiner process W(t) as
        `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
        
        Arguments:
            s0: 원래 주식 가격, 기본값 100
            mu: 주가의 추이(Drift, upwards or downwards), default 1
            sigma: 주가의 변동성
            deltaT: 미래 가격이 계산되는 기간, 주 단위
            dt (optional): 기간의 세분화, default 0.1
        
        Returns:
            s: 기간 deltaT에 걸쳐 시뮬레이션된 주가가 포함된 NumPy array
        """
        n_step = int(deltaT/dt)
        time_vector = np.linspace(0,deltaT,num=n_step)
        # Stock variation
        stock_var = (mu-(sigma**2/2))*time_vector
        # Forcefully set the initial value to zero for the stock price simulation
        self.x0=0
        # Weiner process (calls the `gen_normal` method)
        weiner_process = sigma*self.gen_normal(n_step)
        # Add two time series, take exponent, and multiply by the initial stock price
        s = s0*(np.exp(stock_var+weiner_process))
        
        return s

In [ ]:
b = Brownian()
for i in range(4):
    plt.plot(b.gen_random_walk(1000))
plt.show()

In [ ]:
b = Brownian(20)
for i in range(4):
    plt.plot(b.gen_normal(1000))
plt.show()